In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
# Read the data
df = pd.read_csv('training/init_preprocessed_data_without_index.csv')
len(df)

d:\School\University\3A\CS_492\AI-Bias-Experiment\models\training\venv\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (37,47,48,49,50,51,52,53,54,55,56,57,58,59,60,76,77) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


3252599

In [3]:
df.columns

Index(['SERIAL', 'PERNUM', 'HHWT', 'CLUSTER', 'STRATA', 'PERWT', 'AGE',
       'YRMARR', 'YRNATUR', 'RACNUM', 'DEGFIELD', 'DEGFIELDD', 'OCC2010',
       'WKSWORK1', 'UHRSWORK', 'INCWAGE_CPIU_2010', 'PWSTATE2', 'PWCOUNTY',
       'PWTYPE', 'TRANTIME', 'isFemale', 'isAmericanIndian', 'isAsian',
       'isBlack', 'isPacificIslander', 'isWhite', 'isOtherRace',
       'hasPrivateHealthInsurance', 'hasEmployerHealthInsurance',
       'hasPurchasedPrivHealthInsurance', 'hasMilitaryHealthInsurance',
       'hasPublicHealthInsurance', 'hasMedicare', 'hasMedicaid',
       'hasVeteransHealthInsurance', 'hasIndianHealthInsurance',
       'hasHealthInsurance', 'isInSchool', 'carpools', 'isHispanic',
       'sameSexMarriage', 'mixedRaceMarriage', 'isGroupQuarters', 'bornInUS',
       'isMarried', 'wasMarried', 'neverMarried', 'speaksEnglish',
       'speaksOnlyEnglish', 'speaksEnglishWell', 'noSchooling', 'maxGrade4',
       'maxGrade8', 'maxSomeHS', 'highSchoolDiploma', 'someCollege',
       'assoc

In [4]:
df.head()

,SERIAL,PERNUM,HHWT,CLUSTER,STRATA,PERWT,AGE,YRMARR,YRNATUR,RACNUM,...,isUnpaidFamilyWorker,employedLastYear,employed1to5YrsAgo,unemployedLast5Yrs,commutePrivateVehicle,commutePublicTransportation,commuteBikeOrWalk,workFromHome,attendingPublicSchool,attendingPrivateSchool
0,1,1,13.0,2021000000011,80001,13.0,85,1971.0,NaN,1,...,NaN,False,False,True,NaN,NaN,NaN,NaN,False,False
1,2,1,51.0,2021000000021,80001,51.0,67,1970.0,NaN,1,...,NaN,False,False,True,NaN,NaN,NaN,NaN,False,False
2,3,1,17.0,2021000000031,120001,17.0,74,1991.0,NaN,1,...,NaN,False,False,True,NaN,NaN,NaN,NaN,False,False
3,4,1,61.0,2021000000041,170001,61.0,16,NaN,NaN,1,...,NaN,False,False,True,NaN,NaN,NaN,NaN,False,True
4,5,1,15.0,2021000000051,50001,15.0,83,2016.0,NaN,1,...,NaN,False,False,True,NaN,NaN,NaN,NaN,False,False


Create degree fields

In [5]:
with(open('ipums_fields/degField.json')) as f:
    deg_mapping = json.load(f)

In [6]:
# Make sure each degree field is in the mapping
assert(sorted(df.DEGFIELD.unique().tolist()) == sorted([int(k) for k in deg_mapping.keys()]))

In [7]:
cols_created = 0
for (key, value) in deg_mapping.items():
    # Skip N/A column (may want to fill this with NaN later)
    if value == 'N/A':
        continue

    degName = value.replace(' ', '_').replace(',', '')
    df[f"hasDegree_{degName}"] = (df.DEGFIELD == int(key))
    cols_created += 1

print(f"Created {cols_created} columns")

Created 37 columns


In [8]:
df.drop(columns=['DEGFIELD', 'DEGFIELDD'], inplace=True)

Create occupation fields

In [9]:
with(open('ipums_fields/occupation2010.json')) as f:
    occ_mapping = json.load(f)

In [10]:
# Make sure each degree field is in the mapping
assert(sorted(df.OCC2010.unique().tolist()) == sorted([int(k) for k in occ_mapping.keys()]))

In [11]:
cols_created = 0
for (key, value) in occ_mapping.items():
    # Skip N/A column (may want to fill this with NaN later)
    if value == 'N/A':
        continue

    occName = value.replace(' ', '_').replace(',', '')
    df[f"occupation_{occName}"] = (df.OCC2010 == int(key))
    cols_created += 1

print(f"Created {cols_created} columns")

d:\School\University\3A\CS_492\AI-Bias-Experiment\models\training\venv\lib\site-packages\ipykernel_launcher.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


Created 427 columns


In [12]:
df.drop(columns=['OCC2010'], inplace=True)

Create state fields

In [13]:
with(open('ipums_fields/stateField.json')) as f:
    state_mapping = json.load(f)

In [14]:
df.PWSTATE2.unique()

array([ 0,  1, 21, 39, 12, 47, 13, 28, 22, 86, 26, 17,  6, 55, 48, 27, 51,
       84, 45,  4, 34, 41, 37, 20,  8, 42,  5,  2, 40, 16, 54, 18, 29, 30,
       83, 19, 36, 49, 53, 15, 33, 35, 32, 31, 38,  9, 81, 11, 24, 23, 56,
       82, 85, 88, 25, 10, 46, 44, 50, 72], dtype=int64)

In [15]:
# Make sure each degree field is in the mapping
stateKeys = sorted([int(k) for k in state_mapping.keys()])
# Note that stateKeys doesn't include foreign countries
assert(sorted([val for val in df.PWSTATE2.unique().tolist() if val <= stateKeys[-1]]) == stateKeys)

In [16]:
df["worksOutsideUS"] = (df.PWSTATE2 > stateKeys[-1])
cols_created = 1
for (key, value) in state_mapping.items():
    # Skip N/A column (may want to fill this with NaN later)
    if value == 'N/A':
        continue

    stateName = value.replace(' ', '_')
    df[f"worksIn_{stateName}"] = (df.PWSTATE2 == int(key))
    cols_created += 1

print(f"Created {cols_created} columns")

Created 53 columns


In [17]:
df.drop(columns=['PWSTATE2', 'PWCOUNTY', 'PWTYPE'], inplace=True)

In [ ]:
df.reset_index().to_csv('large_field_preprocessed_data.csv', index=False)